In [1]:
import pandas as pd
import os
# Set working directory
os.chdir(r"..") # should be the git repo root directory
print("Current working directory: " + os.getcwd())
repo_name = 'net-load-forecasting'
assert os.getcwd()[-len(repo_name):] == "net-load-forecasting", "Working directory is not the git repo root directory"


from utils.utils import *

Current working directory: c:\Users\nik\Desktop\Berkeley_Projects\net-load-forecasting


In [11]:
clean_data_path = os.path.join(os.getcwd(),'data','clean_data')

model_data_path = os.path.join(os.getcwd(),'data','model_data')

scenarios_to_build = [
                        'META-1',
                        # 'META-2',
                        # 'META-3',
                        # 'META-4', 
                        # 'META-5'
                    ]

resolutions = [
                '1min', 
                #'15min'
                ]


In [12]:
store = pd.HDFStore(os.path.join(model_data_path, "pv_model_results_2.h5"))

for temp_resolution in resolutions:

    df = pd.read_hdf(os.path.join(clean_data_path, "data_net_load_forecasting.h5"), key=f"{temp_resolution}/netload")

    df_pv = pd.read_hdf(os.path.join(clean_data_path, "data_net_load_forecasting.h5"),  key=f"{temp_resolution}/community_pv")

    df_meta = pd.read_hdf(os.path.join(clean_data_path, "data_net_load_forecasting.h5"), key='pv_metadata')

    df_irr = pd.read_hdf(os.path.join(clean_data_path, "data_net_load_forecasting.h5"), key= f"{temp_resolution}/weather")
    df_irr.rename({'temperature': 'temp_air'}, axis=1, inplace=True)


    list_meta_per_scenario = {}
    for meta_scen in scenarios_to_build:
        list_meta_per_scenario[meta_scen] = prepare_metadata_per_scenario(df, df_meta, df_irr, meta_data_scenario=meta_scen)

    for meta_scen in scenarios_to_build:
        print(f'Generating PV forecasts for scenario: {meta_scen}')
        list_meta = list_meta_per_scenario[meta_scen]
        pv_forecast_per_scenario = []
        for df_meta in list_meta:
            df_pv_forecast = df_meta.apply(lambda x: physical_profile(x, df_irr), axis=1).T.sum(axis=1).to_frame('pv_forecast')
            pv_forecast_per_scenario.append(df_pv_forecast)
        
        df_pv_forecast_per_scenario = pd.concat(pv_forecast_per_scenario, axis=1).mean(axis=1).to_frame(f'pv_forecast_{meta_scen}')
        store.put(f'{temp_resolution}/pv_forecast_{meta_scen}', df_pv_forecast_per_scenario, format='table')

store.close()



Generating PV forecasts for scenario: META-1


c:\Users\nik\miniconda3\envs\gpu2\lib\site-packages\tables\path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'pv_forecast_META-1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
